### Import Library 

In [2]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords 
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import TweetTokenizer

### Data 

In [3]:
data = pd.read_csv("data.csv", encoding='utf-8') #memanggil data
data #menampilkan data

,No,user_name,text,label
0,1,Arnold Mamesah MA,Fobia #COVID19 muncul bersamaan dgn #depresi #...,positif
1,2,Himmah Online,[Berita]\nProgram vaksinasi masih menjadi kont...,positif
2,3,Inaz Putri,"jangan sampai kecolongan, yuk kita terus menja...",positif
3,4,Dicky Syaiful Anwar,Ide sih untuk yg masih menolak di vaksin. Demi...,positif
4,5,Karya Untuk Indonesia,"Halo Sobat Kui, vaksinasi bisa berpengaruh unt...",positif
...,...,...,...,...
4636,4637,Dodok,KAMI tidak mencerminkan gerakan moral #KAMI #O...,negatif
4637,4638,Dodok,Isinya hanya orang-orang provokator #KAMI #Ots...,negatif
4638,4639,Dodok,Krlompok kami adalah orang-orang yg mencoba un...,negatif
4639,4640,Joji,Isinya hanya orang-orang provokator #KAMI #Ots...,negatif


In [4]:
print (data['text'][1]) #melihat isi data secara spesifik pada kolom text baris 1

[Berita]
Program vaksinasi masih menjadi kontroversial di khalayak umum terkait pengaruh efikasi (kemanjuran secaraâ€¦ https://t.co/fNMklyL9aW


### Casefolding (Preprocessing)

In [5]:
#proses casefolding mengubah data uppercase menjadi lower case
def casefolding (tweet):
    tweet = tweet.lower()
    tweet = tweet.strip(" ")
    tweet = re.sub(r' [?|$|.|!2_:")(-+,]', ' ', tweet)
    return tweet

data['lowercase'] = data['text'].apply(casefolding) #membuat kolom baru dengan nama kolom lowercase untuk hasil casefolding
data #menampilkan data
#print (data['text'][0])

,No,user_name,text,label,lowercase
0,1,Arnold Mamesah MA,Fobia #COVID19 muncul bersamaan dgn #depresi #...,positif,fobia #covid19 muncul bersamaan dgn #depresi #...
1,2,Himmah Online,[Berita]\nProgram vaksinasi masih menjadi kont...,positif,[berita]\nprogram vaksinasi masih menjadi kont...
2,3,Inaz Putri,"jangan sampai kecolongan, yuk kita terus menja...",positif,"jangan sampai kecolongan, yuk kita terus menja..."
3,4,Dicky Syaiful Anwar,Ide sih untuk yg masih menolak di vaksin. Demi...,positif,ide sih untuk yg masih menolak di vaksin. demi...
4,5,Karya Untuk Indonesia,"Halo Sobat Kui, vaksinasi bisa berpengaruh unt...",positif,"halo sobat kui, vaksinasi bisa berpengaruh unt..."
...,...,...,...,...,...
4636,4637,Dodok,KAMI tidak mencerminkan gerakan moral #KAMI #O...,negatif,kami tidak mencerminkan gerakan moral #kami #o...
4637,4638,Dodok,Isinya hanya orang-orang provokator #KAMI #Ots...,negatif,isinya hanya orang-orang provokator #kami #ots...
4638,4639,Dodok,Krlompok kami adalah orang-orang yg mencoba un...,negatif,krlompok kami adalah orang-orang yg mencoba un...
4639,4640,Joji,Isinya hanya orang-orang provokator #KAMI #Ots...,negatif,isinya hanya orang-orang provokator #kami #ots...


### Filtering (Preprocessing) 

In [6]:
#menghapus mention, hastag, link, font karakter, double spasi, enter, tab dll
def remove(tweet):
    tweet = re.sub('[0-9]+', '', tweet) #remove angka 
    tweet = re.sub(r'\$\w*', '', tweet)  # remove stock market tickers like $GE
    tweet = re.sub(r'^RT[\s]+', '', tweet) # remove old style retweet text "RT"
    tweet = re.sub(r',','',tweet) # remove koma 
    tweet = re.sub(r'#[\w]*', ' ', tweet) # remove hashtags (only removing the hash # sign from the word)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet) #remove hyperlink
    tweet = re.sub(r'@[\w]*', '', tweet) #remove mention @
    tweet = re.sub(r'  ', '', tweet) #remove doublespasi
    tweet = re.sub(r'   ', '', tweet) #remove triplespasi
    tweet = re.sub(r'    ', '', tweet)
    tweet = re.sub(r'\n+', '', tweet) #remove enter
    tweet = re.sub(r'\n', '', tweet) #remove enter
    tweet = re.sub(r'\t', '', tweet) #remove tab
    tweet = re.sub(r'\t\t\t', '', tweet) #remove tab
    tweet = re.sub(r'[?|$|.|!_:")(-+,€¦=;}{~`â™§‡†ðÿ’ªðÿ™¬ï¼‰š\?><ðŸ’‰˜ž‘“.&-]', '', tweet) #remove karakter
    tweet = tweet.encode('ascii', 'replace').decode('ascii')
    tweet = re.sub(r'·+', '', tweet)
    return tweet

data['remove'] = data['lowercase'].apply(lambda x: remove(x)) #membuat kolom baru dengan nama remove untuk hasil remove
data.sort_values("No", inplace = True, axis = 0)
data.drop_duplicates(subset ="user_name", keep = 'first', inplace = True) #remove duplicate data (tweet ganda) berdasarkan username
print (data['remove'][2174])
print ("Data 132 ", data['remove'][132])
print ("Data 132 ", data['lowercase'][132])
data

?positif covid setelah vaksinasi?
Data 132  
Data 132  https://t.co/slbzj0n3ew, #kediriâ€“ alur pelayanan #vaksinasi covid-19 kota #kediri, dilakukan mulai dari tahap pendaâ€¦ https://t.co/yetlhzyqpp


,No,user_name,text,label,lowercase,remove
0,1,Arnold Mamesah MA,Fobia #COVID19 muncul bersamaan dgn #depresi #...,positif,fobia #covid19 muncul bersamaan dgn #depresi #...,fobia muncul bersamaan dgn dlm berbagai akib
1,2,Himmah Online,[Berita]\nProgram vaksinasi masih menjadi kont...,positif,[berita]\nprogram vaksinasi masih menjadi kont...,[berita]program vaksinasi masih menjadi kontro...
2,3,Inaz Putri,"jangan sampai kecolongan, yuk kita terus menja...",positif,"jangan sampai kecolongan, yuk kita terus menja...",jangan sampai kecolongan yuk kita terus menjag...
3,4,Dicky Syaiful Anwar,Ide sih untuk yg masih menolak di vaksin. Demi...,positif,ide sih untuk yg masih menolak di vaksin. demi...,ide sih untuk yg masih menolak di vaksin demi ...
4,5,Karya Untuk Indonesia,"Halo Sobat Kui, vaksinasi bisa berpengaruh unt...",positif,"halo sobat kui, vaksinasi bisa berpengaruh unt...",halo sobat kui vaksinasi bisa berpengaruh untu...
...,...,...,...,...,...,...
4621,4622,akusimus',Sekarang uni eropa ketat mengontrol vaccine pf...,negatif,sekarang uni eropa ketat mengontrol vaccine pf...,sekarang uni eropa ketat mengontrol vaccine pf...
4628,4629,prasthink,Coroooonaaa #corona #covid19 #c19 #pusing #bog...,negatif,coroooonaaa #corona #covid19 #c19 #pusing #bog...,coroooonaaancovid
4630,4631,Fauzan Rahardian,"wtf, katanya ga bakal berubah? https://t.co/DF...",negatif,"wtf, katanya ga bakal berubah? https://t.co/df...",wtf katanya ga bakal berubah
4631,4632,Discovery â˜¹ðŸ”«,Gaada tanadaÂ² jadi war hammer titan\n#vaksin ...,negatif,gaada tanadaâ² jadi war hammer titan\n#vaksin ...,gaada tanada? jadi war hammer titan


In [7]:
print ("data ke 132", data['remove'][132]) #melihat isi data secara spesifik pada kolom text baris 1

data ke 132 


In [8]:
data.isna().sum() #melihat missing value

No           0
user_name    0
text         0
label        0
lowercase    0
remove       0
dtype: int64

### Import Stemmer (Preprocessing) 

In [9]:
from nltk.corpus import stopwords 
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

stopwords_indonesia = stopwords.words('indonesian')
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)


In [10]:
#melakukan pembersihan ulang pada data serta (remove stopword, emoticon, punctuation, stemming word dan tokenizing)
def clean_tweets(tweet):   
    tweet = re.sub(r'\$\w*', '', tweet) # remove stock market tickers like $GE
    tweet = re.sub(r'^RT[\s]+', '', tweet) # remove old style retweet text "RT"
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)# remove hyperlinks
    tweet = re.sub(r'#', '', tweet) # remove hashtags (only removing the hash # sign from the word)   
    tweet = re.sub(r',','',tweet) #remove koma
    #tweet = re.sub(r'.','',tweet) #remove titik
    tweet = re.sub('[0-9a-z #+_]]+', ' ', tweet) #remove angka
    tweet = re.sub(r'  ', '', tweet) #remove doublespasi
    tweet = re.sub(r'\n', '', tweet) #remove enter
    tweet = re.sub(r'\t+', '', tweet) #remove tab
    tweet = re.sub(r'\t\t', '', tweet)
    tweet = re.sub(r'[?|$|.|!2_:")(-+,€¦=;}{~`â™§‡†ðÿ’ªðÿ™¬ï¼‰š\?><¡ãœ…¢¿¤´´´/''—–£²¨]', '', tweet) #remove karakter  
    
 
    #tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
 
    tweets_clean = []    
    for word in tweet_tokens:
        if (word not in stopwords_indonesia and #remove stopwords
              word not in emoticons and #remove emoticons
                word not in string.punctuation): #remove punctuation
            tweets_clean.append(word)
            stem_word = stemmer.stem(word) #stemming word
           
            
    tweet = ''.join(word for word in tweet.split() if word not in stopwords_indonesia)
    #print (tweet)
    return tweets_clean

data['tweet_clean'] = data['remove'].apply(lambda x: clean_tweets(x))
data.sort_values("No", inplace = True)
data.drop_duplicates(subset ="remove", keep = 'first', inplace = True) #remove duplicate data (tweet ganda)
data.dropna(axis=0)
data.drop(data.remove[[]], axis=0, inplace=True)
data

#print (data['tweet_clean'][1])

,No,user_name,text,label,lowercase,remove,tweet_clean
0,1,Arnold Mamesah MA,Fobia #COVID19 muncul bersamaan dgn #depresi #...,positif,fobia #covid19 muncul bersamaan dgn #depresi #...,fobia muncul bersamaan dgn dlm berbagai akib,"[fobia, muncul, bersamaan, dgn, dlm, akib]"
1,2,Himmah Online,[Berita]\nProgram vaksinasi masih menjadi kont...,positif,[berita]\nprogram vaksinasi masih menjadi kont...,[berita]program vaksinasi masih menjadi kontro...,"[berit, program, vaksinasi, kontroversial, kha..."
2,3,Inaz Putri,"jangan sampai kecolongan, yuk kita terus menja...",positif,"jangan sampai kecolongan, yuk kita terus menja...",jangan sampai kecolongan yuk kita terus menjag...,"[kecolongan, yuk, menjaga, kesehatan, progaram..."
3,4,Dicky Syaiful Anwar,Ide sih untuk yg masih menolak di vaksin. Demi...,positif,ide sih untuk yg masih menolak di vaksin. demi...,ide sih untuk yg masih menolak di vaksin demi ...,"[ide, sih, yg, menolak, vaksin, kebaikan, besa..."
4,5,Karya Untuk Indonesia,"Halo Sobat Kui, vaksinasi bisa berpengaruh unt...",positif,"halo sobat kui, vaksinasi bisa berpengaruh unt...",halo sobat kui vaksinasi bisa berpengaruh untu...,"[halo, sobat, kui, vaksinasi, berpengaruh, pem..."
...,...,...,...,...,...,...,...
4621,4622,akusimus',Sekarang uni eropa ketat mengontrol vaccine pf...,negatif,sekarang uni eropa ketat mengontrol vaccine pf...,sekarang uni eropa ketat mengontrol vaccine pf...,"[uni, eropa, ketat, mengontrol, vaccine, pfize..."
4628,4629,prasthink,Coroooonaaa #corona #covid19 #c19 #pusing #bog...,negatif,coroooonaaa #corona #covid19 #c19 #pusing #bog...,coroooonaaancovid,[corooonaaancovid]
4630,4631,Fauzan Rahardian,"wtf, katanya ga bakal berubah? https://t.co/DF...",negatif,"wtf, katanya ga bakal berubah? https://t.co/df...",wtf katanya ga bakal berubah,"[wtf, ga, berubah]"
4631,4632,Discovery â˜¹ðŸ”«,Gaada tanadaÂ² jadi war hammer titan\n#vaksin ...,negatif,gaada tanadaâ² jadi war hammer titan\n#vaksin ...,gaada tanada? jadi war hammer titan,"[gaada, tanada, war, hammer, titan]"


In [11]:
print ("data ke 132", data['lowercase'][132]) #melihat isi data secara spesifik pada kolom text baris 1

data ke 132 https://t.co/slbzj0n3ew, #kediriâ€“ alur pelayanan #vaksinasi covid-19 kota #kediri, dilakukan mulai dari tahap pendaâ€¦ https://t.co/yetlhzyqpp


In [12]:
data.isna() #melihat missing value

,No,user_name,text,label,lowercase,remove,tweet_clean
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
4621,False,False,False,False,False,False,False
4628,False,False,False,False,False,False,False
4630,False,False,False,False,False,False,False
4631,False,False,False,False,False,False,False


In [13]:
data.isna().sum() #melihat jumlah missing value

No             0
user_name      0
text           0
label          0
lowercase      0
remove         0
tweet_clean    0
dtype: int64

### Convert Data ke CSV 

In [13]:
data.to_csv('data_clean.csv', index=False) #convert data hasil preprocessing ke dalam bentuk file csv
data_clean = pd.read_csv('data_clean.csv', encoding = 'latin1')
data_clean

,No,user_name,text,label,lowercase,remove,tweet_clean
0,1,Arnold Mamesah MA,Fobia #COVID19 muncul bersamaan dgn #depresi #...,positif,fobia #covid19 muncul bersamaan dgn #depresi #...,fobia muncul bersamaan dgn dlm berbagai akib,"['fobia', 'muncul', 'bersamaan', 'dgn', 'dlm',..."
1,2,Himmah Online,[Berita]\nProgram vaksinasi masih menjadi kont...,positif,[berita]\nprogram vaksinasi masih menjadi kont...,[berita]program vaksinasi masih menjadi kontro...,"['berit', 'program', 'vaksinasi', 'kontroversi..."
2,3,Inaz Putri,"jangan sampai kecolongan, yuk kita terus menja...",positif,"jangan sampai kecolongan, yuk kita terus menja...",jangan sampai kecolongan yuk kita terus menjag...,"['kecolongan', 'yuk', 'menjaga', 'kesehatan', ..."
3,4,Dicky Syaiful Anwar,Ide sih untuk yg masih menolak di vaksin. Demi...,positif,ide sih untuk yg masih menolak di vaksin. demi...,ide sih untuk yg masih menolak di vaksin demi ...,"['ide', 'sih', 'yg', 'menolak', 'vaksin', 'keb..."
4,5,Karya Untuk Indonesia,"Halo Sobat Kui, vaksinasi bisa berpengaruh unt...",positif,"halo sobat kui, vaksinasi bisa berpengaruh unt...",halo sobat kui vaksinasi bisa berpengaruh untu...,"['halo', 'sobat', 'kui', 'vaksinasi', 'berpeng..."
...,...,...,...,...,...,...,...
1268,4622,akusimus',Sekarang uni eropa ketat mengontrol vaccine pf...,negatif,sekarang uni eropa ketat mengontrol vaccine pf...,sekarang uni eropa ketat mengontrol vaccine pf...,"['uni', 'eropa', 'ketat', 'mengontrol', 'vacci..."
1269,4629,prasthink,Coroooonaaa #corona #covid19 #c19 #pusing #bog...,negatif,coroooonaaa #corona #covid19 #c19 #pusing #bog...,coroooonaaancovid,['corooonaaancovid']
1270,4631,Fauzan Rahardian,"wtf, katanya ga bakal berubah? https://t.co/DF...",negatif,"wtf, katanya ga bakal berubah? https://t.co/df...",wtf katanya ga bakal berubah,"['wtf', 'ga', 'berubah']"
1271,4632,Discovery Ã¢ËÂ¹Ã°Å¸âÂ«,Gaada tanadaÃÂ² jadi war hammer titan\n#vaksi...,negatif,gaada tanadaÃ¢Â² jadi war hammer titan\n#vaksi...,gaada tanada? jadi war hammer titan,"['gaada', 'tanada', 'war', 'hammer', 'titan']"
